# Use GNU Octave in your C++ program

> *Dec 20, 2019 — Kai T. Ohlhus &lt;<k.ohlhus@gmail.com>&gt; — [CC BY 4.0](http://creativecommons.org/licenses/by/4.0/)*

## Using built-in (C++) Octave functions

In case you want to use some Octave function inside **YOUR** C++ program,
Octave can be included, just like any other programming library.

![img](data/octave_c_cpp_fortran_3.svg)

The easiest task is to include so-called "built-in" functions,
as they are written in C++, too.
**No interpretation of m-code** is necessary.
Additionally, convenient data structures,
such as `Matrix`,
are available to your program.

To find out, if an Octave function is built-in.  Just call, for example:

In [1]:
which norm

'norm' is a built-in function from the file libinterp/corefcn/data.cc


On the other hand, there are "built-in" functions defined in Octave's language
(**m-code/m-files**):

In [2]:
which normest

'normest' is a function from the file /usr/local/share/octave/5.1.0/m/linear-algebra/normest.m


Calling m-files from your C++ program will be explained in the next section
of this notebook.

In the following example,
the norm of some Matrix $A$ shall be computed.
In Octave the code is as simple as:

In [3]:
A = [11, 12; ...
     21, 22]
norm (A)

A =

   11   12
   21   22

ans =  34.495


But for the sake of demonstration,
this computation shall be performed in the following standalone C++ program.

In [4]:
type functions/standalone_builtin.cc

#include <iostream>
#include <octave/oct.h>
#include <octave/builtin-defun-decls.h>

int main () {
  Matrix A(2, 2);

  // Fill matrix
  for (auto j = 0; j < A.columns (); j++) {
    for (auto i = 0; i <  A.rows (); i++) {
      A(i,j) = (i + 1) * 10 + (j + 1);
    }
  }

  // Compute matrix norm
  octave_value_list out = Fnorm (ovl (A), 1);

  // Output results
  std::cout << "Matrix A:"            << std::endl
            << A                      << std::endl
            << "norm (A) = "
            << out(0).double_value () << std::endl;

  return 0;
}



GNU Octave ships with a convenient wrapper program for the system's compiler,
called `mkoctfile`,
to ensure all necessary compiler options.

In this particular case,
the option `--link-stand-alone` has to be passed to ensure
that all dependencies are linked to the resulting binary.
By default `mkoctfile` assumes to compile oct-files,
which are run from within Octave.
In the latter case,
all necessary libraries are already loaded by Octave itself.

In [5]:
mkoctfile --link-stand-alone functions/standalone_builtin.cc -o standalone_builtin.exe

In this particular **Linux** + JuputerLab setup,
GNU Octave is run inside a lightweight and portable
[Singularity](https://sylabs.io/singularity/) container.
This means that the shared libraries are not visible to "the outside world".

Especially, if you run the resulting executable outside the context of the
GNU Octave Singularity container, you will receive errors.
If you are familiar with IPython or Jupyter, prepending an exclamation mark `!`
executes the following statement as system command
and does not use the notebooks kernel (here GNU Octave) to evaluate it.

In [6]:
!./standalone_builtin.exe

./standalone_builtin.exe: error while loading shared libraries: liboctinterp.so.7: cannot open shared object file: No such file or directory



To investigate problems with shared libraries, the **Linux** tool `ldd` is very handy.

In [7]:
!ldd standalone_builtin.exe

	linux-vdso.so.1 (0x00007ffe3a34c000)
	liboctinterp.so.7 => not found
	liboctave.so.7 => not found
	libstdc++.so.6 => /usr/lib64/libstdc++.so.6 (0x00007fe1b7c1b000)
	libgcc_s.so.1 => /lib64/libgcc_s.so.1 (0x00007fe1b7a03000)
	libc.so.6 => /lib64/libc.so.6 (0x00007fe1b7649000)
	libm.so.6 => /lib64/libm.so.6 (0x00007fe1b7311000)
	/lib64/ld-linux-x86-64.so.2 (0x00007fe1b81ab000)



By using Octave's
[`system`-command](https://octave.org/doc/v5.1.0/XREFsystem.html),
one can easily get around this issue for this particular setup.
Thus `system ("ldd standalone_builtin.exe")` will succeed,
but the verbose output is ommitted here.

Finally, we successfully make use of Octave's `norm()`-functions from
within **our own C++-Program** without any interpreter overhead!

**MS Windows** users have to remove the `./`-prefix.

In [8]:
system ("./standalone_builtin.exe")

Matrix A:
 11 12
 21 22

norm (A) = 34.4952
ans = 0


Just one final note to get this right.
We have to use Octave's `system`-command inside this Jupyter-notebook to
invoke `standalone_builtin.exe` correctly.
In a different setup, without JupyterLab, one can simply run the executable
file without any interaction of the Octave interpreter.

## Using ANY Octave function

For some reason,
we want to additionally run some m-file inside our own C++-program.

In the previous example, this was the `normest()`-function.

In [9]:
which normest

'normest' is a function from the file /usr/local/share/octave/5.1.0/m/linear-algebra/normest.m


As C++ is not able to understand m-code.
We need to instantiate the `octave::interpreter` to help us doing this job.

![img](data/octave_c_cpp_fortran_4.svg)

Octave is not a compiled language.
Thus most errors will occur when executing the m-code.
For this reason it is safer to interpret m-code inside a `try...catch`-statement
and to add some exception handling.

The following code looks a little bit bloated,
as we effectively only want to execute a single line of C++-code.
But as said beforehand, most of the code is just added for precaution.
If you do not care about safety or are a perfect programmer by nature,
just three lines of code, shown in the figure above, are absolutely necessary.

In [10]:
type functions/standalone_embedded.cc

#include <iostream>
#include <octave/oct.h>
#include <octave/octave.h>
#include <octave/parse.h>
#include <octave/interpreter.h>

int main (void) {
  Matrix A(2, 2);

  // Fill matrix
  for (auto j = 0; j < A.columns (); j++) {
    for (auto i = 0; i <  A.rows (); i++) {
      A(i,j) = (i + 1) * 10 + (j + 1);
    }
  }

  octave::interpreter interpreter;
  octave_value_list out;

  try {
    int status = interpreter.execute ();
    if (status != 0) {
      std::cerr << "An error occurred!" << std::endl;
      return status;
    }

    out = octave::feval ("normest", ovl (A), 1);

  } catch (const octave::exit_exception& ex) {
    std::cerr << "Octave interpreter exited with status = "
              << ex.exit_status () << std::endl;
  } catch (const octave::execution_exception&) {
    std::cerr << "An error encountered in Octave evaluator!" << std::endl;
  }

  // Output results
  std::cout << "Matrix A:"            << std::endl
            << A                      << std::endl
      

Finally, we compile and run the example:

In [11]:
mkoctfile --link-stand-alone functions/standalone_embedded.cc -o standalone_embedded.exe

**MS Windows** users have to remove the `./`-prefix.

In [12]:
system ("./standalone_embedded.exe")

Matrix A:
 11 12
 21 22

normest (A) = 34.4952
ans = 0
